In [2]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# 村数据不能选的特征['"zccunjitiarea"', '"chengbaodiarea"', '"liangshiarea"', '"year"', '"vid"']
'''
 户数据不能选的特征['"chengbaogd"', '"jygdarea"', '"jycbgdarea"', '"jyczgdarea"', 
 '"jynhgdarea"', '"c201b"', '"zcgdksnum"', '"c202b"', '"zcareahetong"', 
 '"c208b"', '"zcareaqixian"', '"c209b"', '"jitizcarea"', '"c210b"', 
 '"jitizcareacgb"', '"c211b"', '"jitizcareazj"', '"c212b"', '"zcareashouzu"',
 '"c213b"', '"zjxingshi"', '"c214b"', '"c_chengbaogd"', '"x_chengbaogd"', '"s_chengbaogd"', '"cychengbaogd"']
'''

# 村数据流转面积影响因素探究

## 特征选择

In [3]:
# 读取数据
ground_data = pd.read_csv(r'C:\Users\12045\Desktop\地块数据(预处理后).csv')
X = ground_data.drop(columns=['zcgdarea'])
y = ground_data['zcgdarea']

In [4]:
# 方差选择法 (Variance Thresholding)
threshold = 0.5
selector_variance = VarianceThreshold(threshold=threshold)
X_selected_variance = selector_variance.fit_transform(X)
selected_features_variance = X.columns[selector_variance.get_support()]

# 输出特征子集
print("特征子集 - 方差选择法:")
print(selected_features_variance)


特征子集 - 方差选择法:
Index(['hid', 'year', 'chengbaogd', 'chengbaogdks', 'jygdarea', 'jycbgdarea',
       'jyczgdarea', 'jynhgdarea', 'jygdnum', 'jygdnum1', 'jygdnum5',
       'zcqiyearea', 'zchezuoshearea', 'zccunjitiarea', 'zcnonghuarea',
       'zcareahetong', 'zcareaqixian', 'jitizcarea', 'jitizcareacgb', 'zujin',
       'cbdfangwei', 'zrdfangwei', 'cbdarea', 'zrdarea', 'cbdcwjuli',
       'zrdcwjuli', 'cbddljuli', 'zrddljuli', 'zrdyear', 'zrdsyqixian',
       'zrdhetong', 'zrdzuyue', 'd302a', 'd302b', 'd303a', 'd303b', 'd306a',
       'd306b', 'd307a', 'd307b', 'd308a', 'd308b', 'd309b', 'd318a', 'd318b',
       'd327a', 'd327b', 'd330a', 'd330b', 'd332a', 'd332b', 'c_chengbaogd',
       'x_chengbaogd', 's_chengbaogd', 'cychengbaogd'],
      dtype='object')


In [5]:
# L1正则法
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X, y)

# 获取系数非零的特征
selected_features_lasso = X.columns[lasso.coef_ != 0]
# 输出选出的特征 - L1正则化
print("特征子集 - L1正则化:")
print(selected_features_lasso)
print("特征子集列数:", len(selected_features_lasso))

特征子集 - L1正则化:
Index(['hid', 'jygdnum', 'jygdnum1', 'zcqiyearea', 'zccunjitiarea',
       'zcnonghuarea', 'zcareahetong', 'zcareaqixian', 'jitizcarea',
       'jitizcareacgb', 'zrdhetong'],
      dtype='object')
特征子集列数: 11


In [6]:
# 互信息
from sklearn.feature_selection import mutual_info_regression

# 计算特征与目标变量的互信息
mi_scores = mutual_info_regression(X, y)

# 选择互信息得分较高的特征
selected_features_mi = X.columns[np.argsort(mi_scores)[-5:]]
# 输出选出的特征 - 互信息
print("特征子集 - 互信息:")
print(selected_features_mi)
print("特征子集列数:", len(selected_features_mi))

特征子集 - 互信息:
Index(['zujin', 'zcnonghuarea', 'jitizcareacgb', 'zcareahetong', 'jitizcarea'], dtype='object')
特征子集列数: 5


In [7]:
# 创建随机森林回归模型并拟合填充后的数据
model_tree = RandomForestRegressor()
model_tree.fit(X, y)

# 获取特征重要性
feature_importances = model_tree.feature_importances_

# 选择重要性较高的特征
threshold_tree = 0.01  # 假设选择重要性大于0.01的特征
selected_features_tree = X.columns[feature_importances > threshold_tree]

# 输出特征子集
print("\n特征子集 - 基于随机森林的方法:")
print(selected_features_tree)

# 输出特征子集的列数
print("特征子集列数:", len(selected_features_tree))



特征子集 - 基于随机森林的方法:
Index(['zccunjitiarea', 'zcnonghuarea', 'zcareahetong', 'jitizcarea',
       'jitizcareacgb', 'zujin', 'shifouzc', 'cychengbaogd'],
      dtype='object')
特征子集列数: 8


In [8]:
# 使用相关系数法进行特征选择
# 这里以绝对值相关系数大于阈值作为特征选择标准
threshold_correlation = 0.5  # 设定阈值
correlation_matrix = X.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
selected_features_correlation = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold_correlation)]
print("\n特征子集 - 基于相关系数法的方法:")
print(selected_features_correlation)
print("特征子集列数:", len(selected_features_correlation))


特征子集 - 基于相关系数法的方法:
['jynhgdarea', 'jygdnum5', 'zcareahetong', 'jitizcarea', 'jitizcareacgb', 'zrdfangwei', 'cbdcwjuli', 'zrdcwjuli', 'zrdgaosulu', 'zrdguangai', 'zrdfeili', 'zrdwrxiufu', 'shifouzc', 'zrdsyqixian', 'zrdjyptreason2', 'd303b', 'd306b', 'd307b', 'd308b', 'd309b', 'd312b', 'd314a', 'd314b', 'd315b', 'd316b', 'd317a', 'd317b', 'd320a', 'd322a', 'd322b', 'd323a', 'd323b', 'd324a', 'd324b', 'd325a', 'd327b', 'd328b', 'd329b', 'd330b', 'd332b', 'd335b', 'x_chengbaogd', 's_chengbaogd', 'cychengbaogd']
特征子集列数: 44


In [9]:
# 将所有特征子集整合到一个集合中
all_feature_sets = []

all_feature_sets.extend(selected_features_variance)
all_feature_sets.extend(selected_features_tree)
all_feature_sets.extend(selected_features_lasso)
all_feature_sets.extend(selected_features_correlation)
all_feature_sets.extend(selected_features_mi)

# 计算每个特征子集的重叠次数
overlap_counts = {}
for feature_set in all_feature_sets:
    if feature_set in overlap_counts:
        overlap_counts[feature_set] += 1
    else:
        overlap_counts[feature_set] = 1

# 找到重叠次数大于等于1的特征子集
min_overlap_count = 1
selected_feature_sets = [feature_set for feature_set, count in overlap_counts.items() if count >= min_overlap_count]

print("重叠次数大于等于2的特征子集:")
for feature_set in selected_feature_sets:
    print(feature_set)
print("重叠次数:", min_overlap_count)



重叠次数大于等于2的特征子集:
hid
year
chengbaogd
chengbaogdks
jygdarea
jycbgdarea
jyczgdarea
jynhgdarea
jygdnum
jygdnum1
jygdnum5
zcqiyearea
zchezuoshearea
zccunjitiarea
zcnonghuarea
zcareahetong
zcareaqixian
jitizcarea
jitizcareacgb
zujin
cbdfangwei
zrdfangwei
cbdarea
zrdarea
cbdcwjuli
zrdcwjuli
cbddljuli
zrddljuli
zrdyear
zrdsyqixian
zrdhetong
zrdzuyue
d302a
d302b
d303a
d303b
d306a
d306b
d307a
d307b
d308a
d308b
d309b
d318a
d318b
d327a
d327b
d330a
d330b
d332a
d332b
c_chengbaogd
x_chengbaogd
s_chengbaogd
cychengbaogd
shifouzc
zrdgaosulu
zrdguangai
zrdfeili
zrdwrxiufu
zrdjyptreason2
d312b
d314a
d314b
d315b
d316b
d317a
d317b
d320a
d322a
d322b
d323a
d323b
d324a
d324b
d325a
d328b
d329b
d335b
重叠次数: 1
